In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.docstore.document import Document  # Asegúrate de que esta línea esté incluida
from langchain_chroma import Chroma
import re


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
#carga PDF
PATH_PDF = "docs\constitucionar.pdf"
embeddings_model = OpenAIEmbeddings()

loader = PyPDFLoader(PATH_PDF)
documents = loader.load_and_split()

In [4]:
full_text = " ".join([doc.page_content for doc in documents])
full_document = Document(page_content=full_text)

In [5]:
# Dividir el texto en chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,  # Tamaño del chunk aumentado
    chunk_overlap=500,  # Solapamiento del chunk aumentado
    separators=["Artículo", "Art.", "\n\n"]
)
chunks = text_splitter.split_documents([full_document])


In [6]:
def add_metadata(chunks):
    updated_chunks = []
    for chunk in chunks:
        match = re.search(r'Artículo\s*(\d+)', chunk.page_content)
        if match:
            article_number = match.group(1)
            chunk.metadata = {"article_number": article_number}
        updated_chunks.append(chunk)
    return updated_chunks

chunks = add_metadata(chunks)

In [ ]:
# Verificar los chunks generados
for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}:")
    print(chunk.page_content)
    print("\n" + "-"*80 + "\n")

In [8]:
vectorstore = Chroma.from_documents(
    documents=chunks,  # Usar los chunks generados
    embedding=embeddings_model,
    persist_directory="docs/Chroma"
)